決策樹需要計算 **information gain** 

$$\text{Information Gain} = H(p_1^\text{node})- \left(w^{\text{left}}H\left(p_1^\text{left}\right) + w^{\text{right}}H\left(p_1^\text{right}\right)\right),$$

其中 $H$ 是熵

$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$

當$p=0$或$p=1$時熵為0，$p=0.5$時熵為1。

熵表示一個事件的可預測程度。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def computeEntropy(y):
    if len(y) == 0:
        return 0
        
    p = sum(y) / len(y)
    
    if p == 0 or p == 1:
        return 0
    return -p * np.log2(p) - (1 - p) * np.log2(1 - p)

def computeWeightedEntropy(x, y, leftIdx, rightIdx):
    leftW = len(leftIdx) / len(x)
    rightW = len(rightIdx) / len(x)
    leftH = computeEntropy(y[leftIdx])
    rightH = computeEntropy(y[rightIdx])
    return leftW * leftH + rightW * rightH

def computeInformationGain(x, y, leftIdx, rightIdx):
    rootH = computeEntropy(y)
    childH = computeWeightedEntropy(x, y, leftIdx, rightIdx)
    return rootH - childH

In [3]:
def splitByFeature(x, feature):
    left = np.where(x[:, feature] == 1)[0]
    right = np.where(x[:, feature] == 0)[0]
    return left, right

def buildDecisionTree(x, y, featureSize, tree, maxDepth, currDepth):
    if maxDepth <= currDepth or len(x) <= 0:
        return
    
    maxIG = 0
    bestFeature = 0
    for i in range(featureSize):
        left, right = splitByFeature(x, i)
        ig = computeInformationGain(x, y, left, right)
        if ig >= maxIG:
            maxIG = ig
            bestFeature = i

    tree.append((currDepth, bestFeature))
    
    left, right = splitByFeature(x, bestFeature)
    buildDecisionTree(x[left], y[left], featureSize, tree, maxDepth, currDepth + 1)
    buildDecisionTree(x[right], y[right], featureSize, tree, maxDepth, currDepth + 1)

In [4]:
x = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y = np.array([1,1,0,0,1,0,0,1,1,0])

In [5]:
tree = []
buildDecisionTree(x, y, 3, tree, 2, 0)
print(tree)

[(0, 2), (1, 0), (1, 1)]


In [6]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(x, y) 
pred = model.predict(x)
print(pred)

[1 1 0 0 1 0 0 1 1 0]
